# **Example 6** Validation of ATLID level 2 data

This notebook shows how A-EBD profiles (backscatter, extinction, lidar ratio and depol. ratio) can be compared with ground-based data from `.nc`-files.

In [1]:
!pip install earthcarekit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.4/347.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 124.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 132.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import earthcarekit as eck

## Requirements

For this example you need a A-EBD file from an overpass and the matching ground data (e.g., PollyNET `.nc`-file).

Paste both file paths into the cell below:

In [ ]:
fp_aebd = r"./ECA_EXBA_ATL_EBD_2A_20240902T210023Z_20250721T110708Z_01508B.h5"
fp_ground = r"./2024_09_02_Mon_TJK_00_00_01_2000_2130_profiles.nc"

# Also specify the ground site by either setting its name (string) or creating a custom GroundSite object:
site = "dushanbe"
radius_km = 100.0

## **6.1** Open and view datasets

In [ ]:
print("Open the A-EBD dataset:")
with eck.read_product(fp_aebd) as ds_aebd:
    display(ds_aebd)

print("Open the ground-based dataset:")
with eck.read_nc(fp_ground) as ds_ground:
    display(ds_ground)

In [ ]:
import logging
with eck.read_product(fp_aebd) as ds_aebd:
    print("Create a quicklook of the overpass from the A-EBD file:")
    print("Note: When plotting profiles from A-PRO products, the `ecquicklook` function will automatically plot only the closest profile within the set radius.")
    ql = eck.ecquicklook(
        ds=ds_aebd,
        site=site,
        radius_km=radius_km,
        resolution="low",  # Change the A-EBD resolution as needed
        height_range=(0, 12e3),  # Change the plotting height as needed
        selection_max_time_margin="00:01:00",  # Crop the plot to show only 1 Minute around the overpass start and end times
        logger=logging.getLogger(),  # Optional: Shows plotting progress
        show_steps=True,
        mode="exact",
    )
    print("Save the quicklook:")
    eck.save_plot(ql.fig, filepath="./01508B_20250902_aebd_quicklook.png")

## **6.2** Create backscatter, extinction, lidar ratio and depol. ratio profile plots and comparison statistics

In [ ]:
# Note: Ensure that all your input bsc. and ext. data are given as [m-1sr-1]/[m-1] instead of [Mm-1sr-1]/[Mm-1].
#       You can convert your results (plot and stats) to mega by setting the argument to_mega=True.
with (
    eck.read_product(fp_aebd) as ds_aebd,  # Here you could read other or additional EarthCARE files like A-AER
    eck.read_any(fp_ground) as ds_ground,
):
    results = eck.compare_bsc_ext_lr_depol(
        input_ec=ds_aebd,
        resolution="low",  # Change the A-EBD resolution as needed

        input_ec2=ds_aebd,  # Here from the same A-EBD dataset ...
        resolution2="medium",  # ... the high resolution is taken

        input_ground=ds_ground,
        site=site,
        time_var_ground="start_time",  # Set to the time variable name in the ground-based dataset
        height_var_ground="height",  # Set to the height variable name in the ground-based dataset
        bsc_var_ground=["aerBsc_klett_355", "aerBsc_raman_355"],  # Provide list of variables name from the ground-based dataset
        ext_var_ground=["aerExt_klett_355", "aerExt_raman_355"],
        lr_var_ground=["aerLR_klett_355", "aerLR_raman_355"],
        depol_var_ground=["parDepol_klett_355", "parDepol_raman_355"],
        height_range=(0, 12e3),  # Change the plotting height as needed
        selection_height_range=(2e3, 4e3),  # Change the selected height range to get statistics from as needed or set to None
        value_range_bsc=(0, 8e-6),  # Must be given as [m-1sr-1] and will be auto. converted to [Mm-1sr-1] if to_mega=True
        value_range_ext=(0, 3e-4),  # Must be given as [m-1] and will be auto. converted to [Mm-1] if to_mega=True
        value_range_lr=(0, 100),
        value_range_depol=(0, 0.6),
        to_mega=True,  # e.g., plots [m-1sr-1] as [Mm-1sr-1]
    )
    display(results.stats)

In [ ]:
print("Save the plot:")
eck.save_plot(results.fig, filepath="./01508B_20250902_aebd_vs_polly_profiles.png")

print("Save the stat. results:")
results.stats.to_csv("./01508B_20250902_aebd_vs_polly_profiles.csv", index=False)

## **6.3** Edit ground-based profiles, add errors and other plotting customizations

In [ ]:
import numpy as np
with (
    eck.read_product(fp_aebd) as ds_aebd,
    eck.read_any(fp_ground) as ds_ground,
):
    # Add a new variable to the ground-based dataset and edit it.
    # Here data outside a certain height range is set no NaN:
    h = ds_ground["height"].values
    mask = np.logical_not(
        np.logical_and(
            h >= 1.8e3,
            h <= 4.3e3,
        )
    )
    ds_ground["NEW_aerLR_raman_355"] = ds_ground["aerLR_raman_355"].copy()  # Ensure you copy the variable with `.copy()` otherwise the original will also be overwritten
    ds_ground["NEW_aerLR_raman_355"].values[mask] = np.nan

    results = eck.compare_bsc_ext_lr_depol(
        input_ec=ds_aebd,
        input_ground=ds_ground,
        site=site,
        time_var_ground="start_time",  # Set to the time variable name in the ground-based dataset
        height_var_ground="height",  # Set to the height variable name in the ground-based dataset
        resolution="low",  # Change the A-EBD resolution as needed
        # Instead of giving a single variable name you can give a tuple of 2 variable names, where the first corresponds to the signal and the second to its related error variable name
        bsc_var_ground=[
            ("aerBsc_klett_355", "uncertainty_aerBsc_klett_355"),
            ("aerBsc_raman_355", "uncertainty_aerBsc_raman_355"),
        ],
        ext_var_ground=[("aerExt_klett_355", "uncertainty_aerExt_klett_355"), ("aerExt_raman_355", "uncertainty_aerExt_raman_355")],
        lr_var_ground=[("aerLR_klett_355", "uncertainty_aerLR_klett_355"), "NEW_aerLR_raman_355"],  # Here the new variable is used
        depol_var_ground=[("parDepol_klett_355", "uncertainty_parDepol_klett_355"), ("parDepol_raman_355", "uncertainty_parDepol_raman_355")],
        height_range=(0, 12e3),  # Change the plotting height as needed
        selection_height_range=(2e3, 4e3),  # Change the selected height range to get statistics from as needed or set to None
        show_error_ec=True,  # Plots errors as ribbons for the EarthCARE profiles
        to_mega=True,
        # colors_ec=["ec:earthcare", "ec:purple", "black"],  # Edit colors of EarthCARE profiles
        # colors_ground=["tab:blue", "green", "#ff0000"],  # Edit colors of ground-based profiles
        # show_steps=False,  # Plot height bin centers instead of step function
        # single_figsize=(2, 4),  # You may edit the sub figure sizes
    )
    display(results.stats)